## Intro Select

配列中の $k$ 番目の値を $O(N)$ で検索するアルゴリズム

### アルゴリズム $P$
1. 入力配列を $5$ 個の要素からなるグループに分割（余った要素もグループに）
2. グループそれぞれの中央値を求める
3. $\lceil n/5\rceil$ 個の中央値全体の中央値 $x$ を $P$ を用いて再帰的に検索
4. $x$ 未満と $x$ 以上のブロックに分割し、$k$ 番目の要素があるブロックを決定
5. $k$ 番目の要素があるブロックを $P$ で検索

In [56]:
import random

In [57]:
def partition(arr, pivot):
    p_idx = None
    for i, val in enumerate(arr):
        if val == pivot:
            p_idx = i
    if p_idx is None:
        return
    
    # pivotを末尾に
    arr[p_idx], arr[-1] = arr[-1], arr[p_idx]
    p_idx = -1

    # 順に交換
    for i, val in enumerate(arr[:-1]):
        if val <= pivot:
            p_idx += 1
            arr[p_idx], arr[i] = arr[i], arr[p_idx]
    
    # pivotを戻す
    arr[p_idx + 1], arr[-1] = arr[-1], arr[p_idx + 1]

    return p_idx + 1

In [115]:
def partition(arr, pivot):
    p_cnt = 0
    p_idx = None
    for i, val in enumerate(arr):
        if val == pivot:
            p_idx = i
            p_cnt += 1
    if p_idx is None:
        return
    
    # pivotを末尾に
    arr[p_idx], arr[-1] = arr[-1], arr[p_idx]
    p_idx = 0

    # 順に交換
    for i, val in enumerate(arr[:-1]):
        if val <= pivot:
            arr[p_idx], arr[i] = arr[i], arr[p_idx]
            p_idx += 1

    # pivotを戻す
    arr[p_idx], arr[-1] = arr[-1], arr[p_idx]

    # 左、右を調べる
    left = p_idx + 1 - p_cnt
    right = p_idx
    mid = (len(arr) - 1) // 2
    if mid < left:
        idx = left
    elif mid <= right:
        idx = mid
    else:
        idx = right
    
    # pivotを戻す
    arr[idx], arr[p_idx] = arr[p_idx], arr[idx]

    return idx

In [116]:
lst = [5, 3, 7, 5, 3, 1, 2, 2, 3, 4]

i = partition(lst, 2)

print(i, lst)

2 [1, 2, 2, 5, 3, 5, 3, 4, 3, 7]


In [117]:
same = [0] * 10

i = partition(same, 0)

print(i, same)

4 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [118]:
lst = [3, 2, 2, 1, 3, 2, 2]

i = partition(lst, 2)

print(i, lst)

3 [2, 2, 1, 2, 2, 3, 3]


In [119]:
def select(arr, k):
    """
    配列arrのk番目の要素を取得
    """

    if len(arr) == 1:
        return arr[0]

    # ピボット
    pvt = pivot(arr)

    # ピボットのある位置
    pvt_idx = partition(arr, pvt)

    # ピボットの位置で場合分け
    if pvt_idx < k:
        return select(arr[pvt_idx + 1:], k - pvt_idx - 1)
    elif pvt_idx > k:
        return select(arr[:pvt_idx], k)
    else:
        return arr[k]


def pivot(arr):
    """
    arrを5個ずつに分割した配列の中央値の中央値を求める
    """

    if len(arr) == 1:
        return arr[0]

    size = (len(arr) + 4) // 5
    medians = [0] * size

    for i in range(size):
        sub = arr[5 * i: 5 * (i + 1)]

        # 5この要素の中央値
        med = sorted(sub)[(len(sub) - 1) // 2]

        medians[i] = med
    
    # 中央値の中央値を検索
    return select(medians, (size - 1) // 2)

In [120]:
select([1], 0)

1

In [121]:
select([1, 2, 3, 4, 5, 6], 0)

1

In [122]:
select([3, 6, 4, 1, 3, 2, 0], 2)

3

### ランダムテスト

In [66]:
import random

In [67]:
MAX = 1_000_000_000_000_000_000
SIZE = 200_000
REPEAT = 50

In [68]:
median_expected = []
median_actual = []

In [128]:
%%time

random.seed(0)

for _ in range(REPEAT):
    # ランダムなデータの生成
    random_data = [random.randint(0, MAX) for _ in range(SIZE)]

    # ソート
    random_data.sort()

    # 中央値を取得
    med = random_data[SIZE // 2]
    median_expected.append(med)

CPU times: user 5.68 s, sys: 24.9 ms, total: 5.7 s
Wall time: 5.71 s


In [129]:
%%time

random.seed(0)

for _ in range(REPEAT):
    # ランダムなデータの生成
    random_data = [random.randint(0, MAX) for _ in range(SIZE)]

    # 中央値を取得
    med = select(random_data, SIZE // 2)
    median_actual.append(med)    

CPU times: user 8.89 s, sys: 53.4 ms, total: 8.94 s
Wall time: 8.94 s


In [78]:
assert median_expected == median_actual

In [126]:
lst = [1, 5, 4, 6, 3, 2, 1]

ex = sorted(lst)[7 // 2]
ac = select(lst, 7 // 2)

print(lst, sorted(lst))

ex, ac

[1, 1, 4, 6, 3, 2, 5] [1, 1, 2, 3, 4, 5, 6]


(3, 3)

In [127]:
for i in range(7):
    print(select(lst, i))

1
1
2
3
4
5
6


In [123]:
%%time
same_data = [1 for _ in range(1_000)]

select(same_data, 500)

CPU times: user 1.09 ms, sys: 1 µs, total: 1.09 ms
Wall time: 1.1 ms


1